In [1]:
import pandas as pd
import numpy as np

In [2]:
## https://www.servel.cl/centro-de-datos/procesos-electorales-detalle/?id=13

regions = [
    "https://www.servel.cl/wp-content/uploads/2023/06/12_Resultados_Mesa_Concejales_TER-MAGALLANES.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/11_Resultados_Mesa_Concejales_TER-AISEN.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/10_Resultados_Mesa_Concejales_TER-LOS-LAGOS.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/14_Resultados_Mesa_Concejales_TER-LOS-RIOS.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/09_Resultados_Mesa_Concejales_TER-LA-ARAUCANIA.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/08_Resultados_Mesa_Concejales_TER-BIOBIO.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/07_Resultados_Mesa_Concejales_TER-MAULE.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/06_Resultados_Mesa_Concejales_TER-OHIGGINS.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/13_Resultados_Mesa_Concejales_TER_2-METRO-2.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/13_Resultados_Mesa_Concejales_TER_1-METRO-.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/05_Resultados_Mesa_Concejales_TERVALPO.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/04_Resultados_Mesa_Concejales_TERCOQUIMBO.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/03_Resultados_Mesa_Concejales_TERCOPIAPO.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/02_Resultados_Mesa_Concejales_TER-ANTOFA.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/01_Resultados_Mesa_Concejales_TER-IQUIQUE.xlsx",
    "https://www.servel.cl/wp-content/uploads/2023/06/15_Resultados_Mesa_Concejales_TERARICA.xlsx"
]

In [3]:
output = []
for path in regions:
    frag = pd.read_excel(path)
    output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

In [4]:
def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

df["polling_id"] = create_polling_id(df, columns=["Región", "Provincia", "Distrito", "Comuna", "Circunscripción", "Mesa Nº", "Tipo"])

In [9]:
df[df.Candidato.fillna("#").str.contains("BLANCO")].Candidato.unique()

array(['VOTOS EN BLANCO', 'LUIS BLANCO AREVALO',
       'GUILLERMO BLANCO ANTINAO', 'VICTOR PAULINO ANCAN BLANCO',
       'ALEJANDRO HUINCA BLANCO', 'ROBERTO FRANCISCO BLANCO CHIHUAILAF',
       'MARIO ENRIQUE LLANCALEO BLANCO', 'ALBERTO BLANCO JORQUERA',
       'MARIO UNDURRAGA CASTELBLANCO', 'MIGUEL ANGEL SALAZAR BLANCO',
       'FERNANDO CERDA BLANCO', 'MACARENA BLANCO SILVA',
       'MARCELO IVAN BLANCO NAVARRO', 'EDUARDO EBNER BLANCO',
       'DAISY PAOLA GUTIERREZ BLANCO', 'DAVID BLANCO URZUA',
       'LEONOR EMA MUÑOZ BLANCO', 'ANDREA BLANCO SAGREDO',
       'JUAN PABLO BLANCO BLANCO', 'VIRGINIA CHOQUE BLANCO'], dtype=object)

In [12]:
# VOTOS NULOS
# VOTOS EN BLANCO	
# TOTALES
df["Candidato"] = df["Candidato"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS NULOS": "NULL"
})

df_spoilt = df[df["Candidato"].isin(["NULL", "BLANK"])].copy()
df_spoilt = df_spoilt[["Candidato", "Votos TER", "polling_id"]].rename(columns={"Candidato": "candidate", "Votos TER": "value"})
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

df = df[~df["Candidato"].isin(["BLANK", "NULL", "TOTALES", np.nan])].copy()

In [13]:
data = df[["polling_id", "Pacto", "Votos TER"]].dropna(subset=["Pacto"])
data = data.rename(columns={"Pacto": "candidate", "Votos TER": "value"})

tt = data.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt["flag_candidates"] = 1

tt.head()

,polling_id,candidate,value,rate,flag_candidates
0,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-27.0-AISEN...,PACTO ALTERNATIVA DEMOCRATICA,0.0,0.000000,1
1,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-27.0-AISEN...,PACTO AYSEN,1.0,0.010309,1
2,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-27.0-AISEN...,PACTO CHILE VAMOS PRI-EVOPOLI E INDEPENDIENTES,5.0,0.051546,1
3,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-27.0-AISEN...,PACTO CHILE VAMOS RN E INDEPENDIENTES,21.0,0.216495,1
4,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-27.0-AISEN...,PACTO CHILE VAMOS UDI-INDEPENDIENTES,32.0,0.329897,1


In [14]:
data = pd.concat([
    tt,
    df_spoilt
])

In [6]:
df_location = df[["polling_id", "Región", "Provincia", "Distrito", "Comuna", "Circunscripción", "Mesa Nº", "Tipo"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Región": "region",
    "Comuna": "commune",
    "Provincia": "province",
    "Distrito": "district",
    "Circunscripción": "circunscription",
    "Mesa Nº": "polling_station",
    "Tipo": "type"
})

In [15]:
data.to_csv("../data_output/Chile/2016_councilors_polling_station.csv.gz")
df_location.to_csv("../data_output/Chile/2016_councilors_polling_station_location.csv.gz")